In [ ]:
import pandas as pd
import os
import requests
import json
from dfply import *
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

import ipywidgets as widgets

In [ ]:
# Indtast sti til mappe med datafiler. 
# Sørg for, at det kun er de relevante .csv-filer, der er tilstede i mappen. 
path_to_files = './'

In [ ]:
files = os.listdir(path_to_files)

In [ ]:
paths = [os.path.join(path_to_files, x) for x in files if x.endswith(".csv")]

In [ ]:
first = True
for x in paths:
    df_merge = pd.read_csv(x, thousands=".")
    df_merge["kom"] = x.replace(".csv", "").replace(path_to_files, "")
    if first:
        df = df_merge
        first = False
    else:
        df = pd.concat([df, pd.read_csv(x, thousands=".")])
df >>= drop(X.Url)

In [ ]:
df.pct_udvikling = df.pct_udvikling.apply(lambda x: float(x.replace("%", "")))

In [ ]:
df.columns

In [ ]:
df.columns = ['Bogstav', 'Parti', 'Stemmetal', 'udvikling', 'pct_udvikling', 'område', 'ant_stemberet', 'gyldige_stem', 'gyldige_stem_udv', 'kom']

In [ ]:
afst = requests.get('https://api.dataforsyningen.dk/afstemningsomraader')
afst = [[x['kommune']['navn'], x["navn"], x["bbox"]] for x in afst.json()]

# pseudo area = (sw_longitude - ne_longitude)  *  (sw_latitude - ne_latitude)
get_area_prox = lambda x: (x[1]-x[3]) * (x[0]-x[2])
afst = [x+[get_area_prox(x[2])] for x in afst]

df_afst = pd.DataFrame(afst, columns=["kom", "område", "coordinate", "bbox_area"])

df.område = df.område.apply(lambda x: x[:30] if len(x) > 30 else x)
df_afst.område = df_afst.område.apply(lambda x: x[:30] if len(x) > 30 else x)

d = df.merge(df_afst, on=["kom", "område"], how="left")

In [ ]:
d["tæthed"] = d.ant_stemberet/d.bbox_area
d['forrige_valg'] = d.Stemmetal - d.udvikling

# "Bad appels"

Her findes de områder, der ikke har troværdige data fra sidste valg.  
Dette kan skyldeas, at der er tale om nye valgsteder, men det vides ikke med sikkerhed.  
Fælles for disse valgsteder er, at alle partierne i området er gået fra 0 til X antal stemmer ved dette valg.  
Dermed er det en indikation for, at valgstedet ikke har været brugt før. 


In [ ]:
bad_apples = (d.groupby(["kom", "område"]).forrige_valg.sum() >> mask(X == 0)).to_frame().reset_index()
bad_apples_omr = bad_apples.område
d >>= mask(X.område.isin(bad_apples_omr) == False)
print("Følgende valgsteder er blevet fjernet fra datasættet pga. manglende bbox")
bad_apples

In [ ]:
parti_liste = list(d.Bogstav.unique())
parti_liste.sort()

# Generering af figur

Kør første celle nedenfor.  
Dette producerer en dropdown, som kan justeres til forskellige partier.  
Kør derefter de to celler nedenfor for at generere graf og se de valgsteder, hvor partiet har klaret sig bedst. 

In [ ]:
parti_valg = widgets.Dropdown(
    options=parti_liste,
    value='A',
    description='Vælg parti',
    disabled=False)
parti_valg

In [ ]:
parti = parti_valg.value
dA = d >> mask(X.Bogstav == parti)

fig = plt.figure(figsize=(12,6))
ax  = fig.add_subplot(111)

sns.set(font_scale=1.4)
sns.set_style("whitegrid", {"grid.color": ".6", "grid.linestyle": ":"})
ax = sns.scatterplot(data=dA, x="tæthed", y="pct_udvikling", alpha=0.5, size="Stemmetal", sizes = (10,150))
ax.set_position([0.15,0.15,0.58,0.8])
plt.xscale('log')
plt.title(f"{parti}'s ændrede stemmetal på det enkelte valgsted")
plt.xlabel("Befolkningstæthed (bbox proxy)")
plt.ylabel("Udvikling i procentpoint")
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f"{int(x)}%"))
ax.legend(title="Samlede stemmer\npå valgsted", bbox_to_anchor=(1.03, 1), loc=2, borderaxespad=0.)


# Afkommentér for at gemme filen som billede
# plt.savefig(parti+".png")


In [ ]:
øverste_procent = dA.pct_udvikling.quantile(q=0.99)
print("Grænseværdi for øverste percentil:", øverste_procent)

In [ ]:
(
    dA >> mask(X.pct_udvikling > øverste_procent) >> 
    arrange(X.tæthed, ascending=False) >> 
    drop(
        X.forrige_valg,     X.coordinate, 
        X.gyldige_stem_udv, X.bbox_area
    )
)